# COVID-19 World
> Tracking coronavirus total cases, deaths and new cases by country. A detailed view is provided for the Kuwait, GCC, US, Europe

- comments: false
- author: Nabeel Khan
- categories: [COVID-19, Kuwait, GCC, overview, interactive]
- image: images/covid-overview.png
- permalink: /covid-overview/
- hide: false

In [0]:
#hide
print('''
Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/
''')

In [0]:
#hide
import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [0]:
#hide

# FETCH
base_url = 'https://raw.githubusercontent.com/nabeelkhan/COVID19/master/Dashboard/'
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + 'overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [0]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('deaths')
dft_recovered = get_frame('recovered')
LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]


dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_5ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_5ago].sum()

In [5]:
#hide
df_table = (pd.DataFrame(dict(Cases=dfc_cases, Deaths=dfc_deaths, PCases=dfp_cases, PDeaths=dfp_deaths))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
df_table.rename(columns={'index': 'Country/Region'}, inplace=True)
for c in 'Cases, Deaths'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table.head(15)

Country/Region   Cases  Deaths  ...  Deaths (+)  Fatality Rate      Continent
0               US  396223   12722  ...        6796            3.2  North America
1            Spain  141942   14045  ...        3697            9.9         Europe
2            Italy  135586   17127  ...        3212           12.6         Europe
3           France  110065   10343  ...        4945            9.4         Europe
4          Germany  107663    2016  ...         909            1.9         Europe
5            China   82718    3335  ...          13            4.0           Asia
6             Iran   62589    3872  ...         712            6.2           Asia
7   United Kingdom   55949    6171  ...        3245           11.0         Europe
8           Turkey   34109     725  ...         369            2.1           Asia
9      Switzerland   22253     821  ...         285            3.7         Europe
10         Belgium   22194    2035  ...        1024            9.2         Europe
11     Netherlands   19709    2108  ...         767           10.7         Europe
12          Canada   17872     375  ...         236            2.1  North America
13          Brazil   14034     686  ...         362            4.9  South America
14         Austria   12639     243  ...          85            1.9         Europe

[15 rows x 9 columns]

In [6]:
#hide
# Map Kuwait, Bahrain, Qatar, KSA, Oman and UAE
df_table[df_table['Country/Region'].isin(['Kuwait', 'Qatar', 'Bahrain', 'Oman', 'United Arab Emirates', 'Saudi Arabia'])]

Country/Region  Cases  Deaths  ...  Deaths (+)  Fatality Rate  Continent
36          Saudi Arabia   2795      41  ...          20            1.5       Asia
40  United Arab Emirates   2359      12  ...           4            0.5       Asia
44                 Qatar   2057       6  ...           3            0.3       Asia
66               Bahrain    811       5  ...           1            0.6       Asia
68                Kuwait    743       1  ...           1            0.1       Asia
89                  Oman    371       2  ...           1            0.5       Asia

[6 rows x 9 columns]

In [0]:
#hide
# as GCC Continet
# df_table.at([35, 42, 44, 64, 69, 90], 'Continent', 'GCC')

df_table.loc[df_table['Country/Region'].isin(['Kuwait', 'Qatar', 'Bahrain', 'Oman', 'United Arab Emirates', 'Saudi Arabia']), 'Continent'] = 'GCC'

In [8]:
#hide_input
# Check Update
df_table[df_table['Continent']== 'GCC']

Country/Region  Cases  Deaths  ...  Deaths (+)  Fatality Rate  Continent
36          Saudi Arabia   2795      41  ...          20            1.5        GCC
40  United Arab Emirates   2359      12  ...           4            0.5        GCC
44                 Qatar   2057       6  ...           3            0.3        GCC
66               Bahrain    811       5  ...           1            0.6        GCC
68                Kuwait    743       1  ...           1            0.1        GCC
89                  Oman    371       2  ...           1            0.5        GCC

[6 rows x 9 columns]

In [0]:
#hide
#delete problematic countries from table
df_table = df_table[~df_table['Country/Region'].isin(['Cape Verde', 'Cruise Ship', 'Kosovo'])]

In [10]:
#hide
# world, china, europe, us, 
metrics = ['Cases', 'Deaths', 'Cases (+)', 'Deaths (+)']
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')

# GCC
s_gcc = df_table[df_table['Continent'].eq('GCC')][metrics].sum().add_prefix('GCC ')

summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_5ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu, **s_gcc}
summary

{'Cases': 1425926,
 'Cases (+)': 412731,
 'China Cases': 82718,
 'China Cases (+)': 286,
 'China Deaths': 3335,
 'China Deaths (+)': 13,
 'Deaths': 81861,
 'Deaths (+)': 28880,
 'EU Cases': 711417,
 'EU Cases (+)': 187570,
 'EU Deaths': 57428,
 'EU Deaths (+)': 19320,
 'GCC Cases': 9136,
 'GCC Cases (+)': 4062,
 'GCC Deaths': 67,
 'GCC Deaths (+)': 30,
 'US Cases': 396223,
 'US Cases (+)': 152607,
 'US Deaths': 12722,
 'US Deaths (+)': 6796,
 'since': Timestamp('2020-04-02 00:00:00'),
 'updated': Timestamp('2020-04-07 00:00:00')}

In [11]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

1/22/20  1/23/20  1/24/20  ...  4/5/20  4/6/20  4/7/20
Country/Region                             ...                        
Afghanistan           0        0        0  ...      50      18      56
Albania               0        0        0  ...      28      16       6
Algeria               0        0        0  ...      69     103      45
Andorra               0        0        0  ...      35      24      20
Angola                0        0        0  ...       4       2       1

[5 rows x 77 columns]

In [12]:
#hide_input
template = Template(get_template(paths['overview']))
html = template.render(
    D=summary, table=df_table,  # REMOVE .head(20) to see all values
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    np=np, pd=pd, enumerate=enumerate)
HTML(f'<div>{html}</div>')

Visualizations by [Nabeel Khan](https://twitter.com/TheNabeelKhan)[^1]

[^1]: Source: ["COVID-19 Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). [orignal interactive](https://gramener.com/enumter/covid19/)